In [ ]:
import numpy as np
import tensornetwork as tn
import scipy.sparse.linalg as sla

# (2-site) DMRG

In this notebook, we illustrate the necessary concepts to build density matrix renormalization group (DMRG) as a variational technique to find the ground state.
The full DMRG code is available as separate Python code, containing several convenience methods.

## Tensor-train representation of the Hamiltonian

To perform DMRG, we should write the Hamiltonian as a tensor train (also known as MPO).
As the DMRG machinery is basically independent of the Hamiltonian,
we focus on an extremely simple example: the XX-Model.

$$ \hat{H} = \sum_{i=0}^{N-1} [\sigma^-_i \sigma^{+}_{i+1} + \sigma^+_{i}\sigma^-_{i+1}]$$

The XX-Model can be solved exactly by mapping the interacting spins to non-interacting Fermions.

We state the result for the tensor train and verify it afterwards.
We define the following matrices:

$$
H_L = \begin{pmatrix} 1 & 0 & 0 &0 \end{pmatrix}
\quad
H_i = \begin{pmatrix}
    I & σ⁻_i & σ⁺_i &0 \\
    0 & 0 & 0 & σ⁺_i \\
    0 & 0 & 0 & σ⁻_i \\
    0 & 0 & 0 & I
\end{pmatrix}
\quad
H_R = \begin{pmatrix} 0 \\ 0 \\ 0 \\ 1 \end{pmatrix},
$$
where $I$ is the identity matrix.
Let's check the matrix product $H_i H_{i+1}$:

$$
H_i H_{i+1}
= \begin{pmatrix}
 I & \sigma^-_{i+1} & \sigma^{+}_{i+1} & \sigma^-_i \sigma^{+}_{i+1} + \sigma^+_{i}\sigma^-_{i+1}\\
 0 & 0 & 0 & \sigma^+_{i}\\
 0 & 0 & 0 & \sigma^-_{i}\\
 0 & 0 & 0 & I
\end{pmatrix}
$$

We see, that we accumulated the nearest neighbor coupling in the top right corner. This element will be selected by sandwiching the term between $H_L$ and $H_R$.
The first row is now the previous column.

We see the equivalence

$$
\hat{H} = H_L \left[\prod_{i=0}^{N-1} H_i\right] H_R.
$$




In [ ]:
size = 5
phys_dim = 2


def chain(nodes):
    """Connect a chain of nodes 'left' to 'right'."""
    for left, right in zip(nodes[:-1], nodes[1:]):
        tn.connect(left["right"], right["left"])

In [ ]:
MO_AXES = ("left", "right", "phys_in", "phys_out")

s_p = np.sqrt(2) * np.array([[0, 0],
                             [1, 0]])
s_m = s_p.T
idx = np.eye(2)
mat = np.zeros([4, 4, phys_dim, phys_dim])
mat[0, :-1, :, :] = idx, s_m, s_p
mat[1:, -1, :, :] = s_p, s_m, idx
left = np.array([1, 0, 0, 0]).reshape([4, 1, 1])  # left boundary
right = np.array([0, 0, 0, 1]).reshape([4, 1, 1])  # right boundary
nodes = [tn.Node(mat, name="H", axis_names=MO_AXES) for __ in range(size)]
node_l = tn.Node(left, name="LH", axis_names=["right", "phys_in", "phys_out"])
node_r = tn.Node(right, name="HR", axis_names=["left", "phys_in", "phys_out"])

chain([node_l] + nodes + [node_r])
left, ham, right = node_l, nodes, right
tn.to_graphviz([node_l] + nodes + [node_r])

## Tensor-train representation of states

### The bond-dimension as the computational cost

As previously discussed, the tensor train of an state (also known as MPS) can be obtained from repeated SVD of a state.
However, the states are too large to fit in memory and SVD is expensive,
thus, we have to start from an tensor train.
Likewise, an uncompressed tensor train is too large.
We introduce a maximal bond dimension (oftentimes referred to as $\chi$) to limit the size.
For now, we'll assume that we know a state, and check the theoretical bond-dimensions obtained from SVD without any truncation.

SVD decomposes a matrix $M^{m\times n}$ into
$$ M^{m\times n} = U^{m\times k} \Sigma^{k\times k} {(V^\dagger)}^{k \times n}; \qquad k = \min(m, n)$$
(this is the economic version).

We start from a state with $N$ sites, this is a vector of dimension $2^N$.
Splitting the first site, we get a bond of dimension $\chi_1=\min(2, 2^{N-1}) = 2$.
The splitting the next site, well get $\chi_2 = \min(2^2, 2^{N-2}) = 2^2$.
Thus, the bond dimension grows exponentially, reaching its maximum $\chi_{N/2} = \min(2^{N/2}, 2^{N/2}) = 2^{N/2}$ in the middle of the chain.
Unsurprisingly, we gained nothing so far, as we have only rewritten our problem.

We start from an tensor train with (small) bond-dimension and try to optimize it by minimizing the energy.
The bond-dimension will automatically grow, and be truncated to keep reasonable computational cost.

In [ ]:
MS_AXES = ("left", "phys", "right")

max_bond_dim = 10
bond_dims = [1] + [min(2**(site), 2**(size-site), max_bond_dim)
                   for site in range(1, size)] + [1]
rng = np.random.default_rng()
nodes = [
    tn.Node(rng.standard_normal([bond_l, phys_dim, bond_r]),
            axis_names=MS_AXES, name=str(ii))
    for ii, (bond_l, bond_r)
    in enumerate(zip(bond_dims[:-1], bond_dims[1:]))
]
chain(nodes)
state = nodes
tn.to_graphviz(nodes)

### Compression and center of orthogonality


We have already seen, that the SVD provides the *optimal* compression of a matrix.
This can be easily be generalized to tensor networks given above.
If we make a node $A$ the center of orthogonality, the SVD provides the *optimal* compression.

To see this, we note that the Frobenius norm is $\lVert{M}\rVert = \mathrm{Tr}{MM^\dagger}$.
A tensor network in canonical form can be written in form $T = UAV^\dagger$ with $U^\dagger U = V V^\dagger = I$. Thus, we have $\lVert T \rVert = \lVert A \rVert$.

The QR/RQ decomposition is a cheap (at least cheaper than SVD) way, to get the state into orthogonal form. We set site 0 as the center of orthogonality by sweeping from right to left through the chain.

In [ ]:
for pos in range(size-1, 0, -1):
    node_r = state[pos]
    left, state[pos] = tn.split_node_rq(
        node_r, right_name=f"R{pos}",
        left_edges=[node_r['left']],
        right_edges=[node_r['phys'], node_r['right']],
    )
    state[pos].add_axis_names(MS_AXES)
    node_l = nodes[pos-1]
    state[pos-1] = tn.contract_between(node_l, left, name=str(pos-1),
                                     axis_names=MS_AXES)
    # normalize network
    state[pos-1].tensor /= np.linalg.norm(state[pos-1].tensor)
tn.to_graphviz(state)

We prepared some helper libraries to make things easier:

In [ ]:
from pathlib import Path
from sys import path

path.insert(0, str(Path.cwd().parent))

import tensortrain as tt

In [ ]:
state = tt.State.from_random(

    phys_dims=[2]*size,
    bond_dims=[min(2**(site), 2**(size-site), max_bond_dim)
               for site in range(1, size)]
)
print(state)
state.set_center(2)
print(state)
print(state.copy(conjugate=True))
ham = tt.heisenbergxx.xx_hamiltonian(size)

tn.to_graphviz(state)

# Minimization of energy

To find the ground state, we need to minimize the energy.
This is given by the network:

In [ ]:
state.set_center(1)

bra, ham_, ket = state.copy(), ham.copy(), state.copy(conjugate=True)
for site in range(size):
    tn.connect(bra[site]["phys"], ham_[site]["phys_in"])
    tn.connect(ham_[site]["phys_out"], ket[site]["phys"])
tn.connect(bra[-1]["right"], ham_.right["phys_in"])
tn.connect(ket[-1]["right"], ham_.right["phys_out"])

tn.connect(ham_.left["phys_in"], bra[0]["left"])

tn.connect(ham_.left["phys_out"], ket[0]["left"])
tn.to_graphviz([ham_.left] + bra.nodes + ket.nodes + ham_.nodes + [ham_.right])

We know very well, that this energy minimization amounts to finding the smallest eigenvalue, as we have a quadratic form:
$$
\min_{\langle\Psi\vert\Psi\rangle = 1} \langle \Psi \vert \hat{H} \vert \Psi\rangle
= \min_{\Psi^\dagger\Psi = 1}  \Psi^\dagger H  \Psi
= \min_{\Phi^\dagger \Phi = 1} \Phi^\dagger \Lambda \Phi
= \min_{\Phi^\dagger \Phi = 1} \Lambda_{ii} {\vert\Phi_i\vert}^2
= \min \Lambda_{ii}.
$$

Finding the global minimum is, however, not possible.
This would amount to diagonalizing the *full* many-body Hamiltonian.

Instead, we minimize the energy with respect the tensor of only two adjacent sites.


In [ ]:
# minimize for sites 1 and 2
left = ham_.left
for site in range(0, 1):
    left = tn.contractors.auto(
      [left, ham_[site], bra[site], ket[site]],
      output_edge_order=[nodes[site]["right"] for nodes in (ham_, bra, ket)]
    )

left.name = "L"
left.axis_names = ["right", "phys_in", "phys_out"]
right = ham_.right
for site in range(size-1, 2, -1):
    right = tn.contractors.auto(
      [ham_[site], bra[site], ket[site], right],
      output_edge_order=[nodes[site]["left"] for nodes in (ham_, bra, ket)]
    )

right.name = "R"
right.axis_names = ["left", "phys_in", "phys_out"]

eff_ham = [left, *ham_[1:3], right]
tn.to_graphviz(eff_ham)

In [ ]:
left.axis_names

For the Lanczos algorithm, we only need Matrix-Vector products. If we construct the full matrix (by contracting the previous network) the cost is $\mathcal{O}(\chi^4)$ in the bond-dimension $\chi$.
It is cheaper to contract the Matrix-Vector product itself, which can be done with a cost of $\mathcal(\chi^3)$.



In [ ]:
# create linear operator representing the tensor-train matrix

vec_shape = tuple([onode['phys_in'].dimension for onode in eff_ham])
vec_size = np.product(vec_shape)


def matvec(vec):
    """Matrix-vector product of `eff_ham` for Lanczos."""
    vec_node = tn.Node(vec.reshape(vec_shape))
    ttop = tn.replicate_nodes(eff_ham)
    out_edges = [onode["phys_out"] for onode in ttop]
    for ii, onode in enumerate(ttop):
        tn.connect(vec_node[ii], onode['phys_in'])
    res = tn.contractors.auto(ttop + [vec_node], output_edge_order=out_edges)
    return res.tensor.reshape(-1)


eff_ham_operator = sla.LinearOperator(shape=(vec_size, vec_size), matvec=matvec)

# use previous result as starting point
node1, node2 = state.copy()[1:3]

v0 = tn.contract_between(
    node1, node2,
    output_edge_order=[node1["left"], node1["phys"], node2["phys"], node2["right"]]
)

gs_energy, gs_vec = sla.eigsh(eff_ham_operator, k=1, which='SA', v0=v0.tensor.reshape(-1))
dbl_node = tn.Node(gs_vec.reshape(v0.shape))
tn.to_graphviz([dbl_node])

We have locally optimized the state, for two combined nodes representing two neighboring (physical) sites.
To get back to the tensor train where every node has only one physical index, we need to split the Node using SVD.
We note, that optimizing the combined node, can automatically increase the bond dimension, as the bond after splitting as the dimension $\chi = \min(d \chi_L, d \chi_R)$,
where $d$ is the physical dimension and $\chi_{L/R}$ the left/right bond.

Upon splitting the node, we truncate small singular values to limit the bond dimension.
Typically, we also set a hard limit for the maximal bond dimension, to keep the computation time limited.


In [ ]:
left, rs, rvh, trunc_s = tn.split_node_full_svd(
    dbl_node, dbl_node[:2], dbl_node[2:],
    max_singular_values=20, max_truncation_err=1e-6,
    left_name="U", middle_name="Σ", right_name="V†"
)
trunc_weight = np.sqrt(np.sum(trunc_s**2))
print("Truncated weight", trunc_weight)
tn.to_graphviz([left, rs, rvh])

After this step, the node $\Sigma$ containing the singular values is the new center of orthogonality.
Thus, the norm of the state is the square of the sum of the singular values $\lVert \vert \psi \rangle \rVert = \langle \psi \vert \psi \rangle = \sum_i \sigma_i^2$.
Physical states should always be normalized, thus after truncating singular values we should restore the norm.



In [ ]:
if trunc_weight > 0:
    rs.tensor /= np.sum(rs.tensor**2)

Subsequently, we can combine the node $\Sigma$ with the left or right node, to regain the tensor-train representation with one node per physical site.
The new center of orthogonality will be the site we combine the $\Sigma$ node with.

Thus, combining $\Sigma V^\dagger$, we moved the center of orthogonality one site to the right.

In [ ]:
left.add_axis_names(MS_AXES)
right = tn.contract_between(
    rs, rvh, name=str(2), output_edge_order=[rs[0], *rvh[1:]],
    axis_names=MS_AXES,
)
state.set_range(1, 3, [left, right])

state.center += 1

In [ ]:
print(state)

Now we have all ingredients together, we just have to repeat the steps moving from left to right and back, locally optimizing the nodes.
The full implementation is given in `dmrg_tn.py`, we run an example:


In [ ]:
SIZE = 10
MAX_BOND_DIM = 20   # runtime should be have like O(bond_dim**3)
TRUNC_WEIGHT = 1e-4

state = tt.State.from_random(
    phys_dims=[2]*SIZE,
    bond_dims=[min(2**(site), 2**(SIZE-site), 2)
               for site in range(SIZE-1)]
)
ham = tt.heisenbergxx.xx_hamiltonian(len(state))
dmrg = tt.DMRG(state, ham)

eng, err = dmrg.sweep_2site(MAX_BOND_DIM, TRUNC_WEIGHT)
print(f"GS energy after first sweep:  {eng[-1]}. Max. truncated weight {max(err)}.")
eng, err = dmrg.sweep_2site(MAX_BOND_DIM, TRUNC_WEIGHT)
print(f"GS energy after second sweep: {eng[-1]}. Max. truncated weight {max(err)}.")
print(f"Comparison: exact Gs energy:  {tt.heisenbergxx.exact_energy(len(state))}")
print(f"Error:                        {eng[-1] - tt.heisenbergxx.exact_energy(len(state))}")